In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import gudhi as gd  

In [2]:
networks={}
for name in ['PCD']:
    cName = 'Consensus '+name
    networks[cName]=nx.read_edgelist('1 output\\'+cName)
    networks[cName].name=cName
    
[nx.info(G) for G in networks.values()]

["Graph named 'Consensus PCD' with 170 nodes and 1840 edges"]

# Removing one by one.

In [3]:
def removeNodesReturnLCC(G,removeSet):
    newG=G.copy()
    newG.remove_nodes_from(G.nodes&removeSet)
    newG=nx.subgraph(newG,max(nx.connected_components(newG),key=len))
    return newG

def persistantHomologyFromNetworkWithRips(G,name,dim,plot=True):      
    dm=pd.DataFrame(dict(nx.shortest_path_length(G))).sort_index()
    dm=dm[sorted(dm.columns)]
    skeleton = gd.RipsComplex(distance_matrix = dm.values) 

    Rips_simplex_tree = skeleton.create_simplex_tree(max_dimension = dim)
    
    BarCode = Rips_simplex_tree.persistence()
      
    #----Simplexes Resume
    simplexesLen={}
    for simplex,filtration in Rips_simplex_tree.get_filtration():
        if(len(simplex)-1 not in simplexesLen):
            simplexesLen[len(simplex)-1]=1
        else:
            simplexesLen[len(simplex)-1]+=1    

    simplexesResume=''
    for item in simplexesLen.items():
        simplexesResume+=str(item[0])+': '+str(item[1])+' | '
    simplexesResume[0:-3]  
    
    #----Barcode Resume
    
    dimensionsLen={}
    for dim in range(0,dim+1):
        dimensionsLen[dim]=len([bd for bd in BarCode if (bd[0]==dim)])

    dimensionResume=''
    for item in dimensionsLen.items():
        dimensionResume+=str(item[0])+': '+str(item[1])+' | '
    dimensionResume[0:-3]    
            
    #--Plot
    if plot:
        plt.figure(figsize=(10,10),dpi=250)
        gd.plot_persistence_barcode(BarCode,axes=plt.gca());    
        plt.title(name+'\n Simplexes: '+simplexesResume[0:-3]+'\n Numbers of Bars: '+dimensionResume[0:-3],fontsize=9)
        plt.tight_layout()
        plt.savefig('./4 output/Barcode - '+name+' - Rips.png',dpi=500,facecolor='white')
        plt.show()
    
    with open('4 output/'+name+'.pkl',mode='wb') as f:
        pickle.dump(BarCode,f)

In [4]:
%%time
for name,G_original in networks.items():
    for node in sorted(G_original.nodes):
        G=removeNodesReturnLCC(G_original,[node])
        persistantHomologyFromNetworkWithRips(G,name+'-'+node,3,plot=False)

CPU times: total: 4h 46min 9s
Wall time: 4h 46min 18s
